# application.py

In [ ]:
import tensorflow as tf
from flask import Flask, render_template,request
from tensorflow.keras.preprocessing import image
import os



application = Flask(__name__)
APP_ROOT = os.path.dirname(os.path.abspath(__file__))


@application.route('/',methods=['GET','POST'])
def index():
    if request.method == 'POST':
            if 'file' not in request.files:
                return 'there is no image in form!'
            final_data= request.files['file']
            target = os.path.join(APP_ROOT, 'static')
            destination = "/".join([target, final_data.filename])
            final_data.save(final_data.filename)
            final_data.save(destination)
            model = tf.keras.models.load_model('bestmodel_VGG16_512.h5', compile=False)
            test_image = image.load_img(final_data.filename , target_size=(512, 512))
            test_image = image.img_to_array(test_image)
            test_image = test_image / 255
            test_image = tf.expand_dims(test_image, axis=0)
            prediction = model.predict(test_image)
            os.remove(final_data.filename)
            if prediction[0][0]<= 0.5:
                s = prediction[0][0] * 2
                non_Dank_proba = round(abs(1 - s),3)
                if (non_Dank_proba == 0):
                    non_Dank_proba = 1
                Dank_proba=round((1 - non_Dank_proba),3)
            elif prediction[0][0]> 0.5:
                s = prediction[0][0] * 2
                Dank_proba = round(abs(1 - s),3)
                non_Dank_proba = round((1 - Dank_proba),3)
            if prediction[0][0]>0.5:
                prediction='Image is a Dank'
            else:
                prediction='Image is Not a Dank'
            return render_template('results.html',predcited=[prediction,Dank_proba,non_Dank_proba,final_data.filename])
    else:
        return render_template('index.html')

if __name__ == "__main__":
    application.run()

# index.html

In [ ]:


<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">

    <title>Dank or Not? – Analyzing and Predicting the Popularity of Memes on Reddit</title>

</head>
	<style>
    #main-footer
    {
      color: #FFFFFF;
      font-family: "Segoe UI";
      background: #2B2B2B;
      text-align: center;
      margin-top: 125px;
      padding: 16px;
      bottom: 8px;
    }</style>
<body>
<div class="container",class="form-group">
        <h2 >Dank or Not? – Analyzing and Predicting the Popularity of Memes on Reddit</h2>

  <form action = "/" method = "post" enctype="multipart/form-data">
        <img id="blah"  width="300" height="300" alt="meme_image"/>
        <br>
        <input type="file" name="file" onchange="document.getElementById('blah').src = window.URL.createObjectURL(this.files[0])"/>
        <br>
        <br>
        <input type = "submit" value="Predict">
        <br>
    </form>
  <h4>Download sample_data:</h4>
    <br>
    <h5>1.Image link: https://i.redd.it/ecqosvb39zm41.jpg</h5>
    <br>
    <h5>2.Image link: https://i.redd.it/ghi2kxtz76o41.jpg </h5>
    <br>
    <h5>3.Image link: https://i.redd.it/fmk7ialms3n41.jpg</h5>
    <br>
    <h5>4.Image link: https://i.redd.it/xa17g8jovin41.jpg	</h5>
    <br>

    </div>


	<footer id="main-footer">
      Copyright &copy; 2021
    </footer>
</body>
</html>

# result.html

In [ ]:


<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">

    <title>Dank or Not? – Analyzing and Predicting the Popularity of Memes on Reddit</title>

</head>
	<style>
    #main-footer
    {
      color: #FFFFFF;
      font-family: "Segoe UI";
      background: #2B2B2B;
      text-align: center;
      margin-top: 125px;
      padding: 16px;
      bottom: 8px;
    }</style>
<body>
<div class="container",class="form-group">
        <h2 >Dank or Not? – Analyzing and Predicting the Popularity of Memes on Reddit</h2>
        <img src="{{url_for('static', filename=predcited[3])}}"  width="300" height="300" />
        <h2> Predicted meme :- {{predcited[0]}} </h2>
        <h2> Predicted probabilty</h2>
        <h3> Dank :- {{predcited[1]}} </h3>
         <h3> Not a Dank :- {{predcited[2]}} </h3>

         <a href="/"><h5 >Click here to move back to the home page</h5 ></a>

    </div>
</body>
</html>

# Requirements.txt

In [ ]:
absl-py==0.12.0
astunparse==1.6.3
cachetools==4.2.2
certifi==2020.12.5
chardet==3.0.4
click==7.1.2
Flask==1.1.2
flatbuffers==1.12
gast==0.3.3
google-auth==1.30.0
google-auth-oauthlib==0.4.4
google-pasta==0.2.0
grpcio==1.32.0
gunicorn==20.0.4
h5py==2.10.0
idna==2.10
itsdangerous==1.1.0
Jinja2==2.11.3
Keras-Preprocessing==1.1.2
Markdown==3.3.4
MarkupSafe==1.1.1
numpy==1.18.5
oauthlib==3.1.0
opt-einsum==3.3.0
Pillow==8.2.0
protobuf==3.15.8
pyasn1==0.4.8
pyasn1-modules==0.2.8
requests==2.23.0
requests-oauthlib==1.3.0
rsa==4.7.2
scipy==1.4.1
six==1.15.0
tensorboard==2.5.0
tensorboard-data-server==0.6.0
tensorboard-plugin-wit==1.8.0
tensorflow==2.3.1
tensorflow-estimator==2.3.0
termcolor==1.1.0
typing-extensions==3.7.4.3
urllib3==1.25.11
Werkzeug==1.0.1
wincertstore==0.2
wrapt==1.12.1